# 1. Data Collection: Scraping Job Description Data with LangChain

In [3]:
from langchain_community.document_loaders import WebBaseLoader

In [4]:
loader = WebBaseLoader("https://cgi.njoyn.com/CORP/xweb/xweb.asp?NTKN=c&clid=21001&Page=JobDetails&Jobid=J0226-1933&BRID=1278207&lang=1")

In [5]:
page_data = loader.load().pop().page_content

In [6]:
print(page_data)




 
























Careers | CGI.com
















































            Skip to main content
        


























Search careers
My profile
Activate alerts

Visit career sites






 Global careers site


Australia
Belgium
Brasil
Canada
Česká Republika
Danmark
Deutschland
Estonia
España
Finland
France
Hungary
India
Italy
Luxembourg
Malaysia
Maroc
Nederland
Norge
Philippines
Polska
Portugal
Romania
Slovensko
Sverige
United Kingdom
United States
















Contact


ENFRGermanPortugueseSpanishSwedishDutchPolishCzechSlovakNorwegianDanish


















Careers
Search career opportunities


























Create your profile | Sign inActivate career alerts



































Find similar career opportunities






Senior AI Specialist

Category:
Analytics and Emerging Digital Technologies



Main location:


					United Kingdom, UK Wide - Various, Various
				





Position ID:
J0226-1933


Employment Type:
F

# 2. Initialize Llama 3.3 with Groq (LLM Setup)

In [7]:
from langchain_groq import ChatGroq

In [8]:
api_key = "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"

In [9]:
llm = ChatGroq(model ="llama-3.3-70b-versatile",
               api_key=api_key,
               temperature=0)

In [10]:
print(llm)

profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True} client=<groq.resources.chat.completions.Completions object at 0x000002393D00A270> async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002393CDE6900> model_name='llama-3.3-70b-versatile' temperature=1e-08 model_kwargs={} groq_api_key=SecretStr('**********')


In [13]:
response = llm.invoke("About Virat Kohli")
print(response.content)

Virat Kohli is a renowned Indian cricketer and former captain of the Indian national team. He is widely regarded as one of the greatest batsmen of all time. Here are some key facts about Virat Kohli:

**Early Life and Career**

Virat Kohli was born on November 5, 1988, in Delhi, India. He began playing cricket at a young age and quickly rose through the ranks, representing Delhi in various age-group tournaments. He made his first-class debut for Delhi in 2006 and soon caught the attention of national selectors.

**International Career**

Kohli made his international debut for India in 2008, playing a One-Day International (ODI) against Sri Lanka. He quickly established himself as a key player in the Indian team, scoring his first ODI century in 2009. He made his Test debut in 2011 and has since become one of the most successful Test batsmen of all time.

**Achievements and Records**

Virat Kohli has numerous records and achievements to his name, including:

* Fastest Indian to reach 10

# 3. Cleaning and Structuring Job Data using Prompt and Llama 3.3

In [14]:
from langchain_core.prompts import PromptTemplate

In [20]:
prompt_extract = PromptTemplate.from_template(
"""


### Scraped Text from website :
{page_data}

###Instructions:
The scraped text is from the Careers page of Nike
Your Job is to extract job postings & return them in JSON Format containing following keys :
"role" , "experience" , "skills" and "description"

Only return the valid JSON
No ```
### valid JSON in key:value (NO PREAMBLE)






"""


)

In [21]:
type(prompt_extract)

langchain_core.prompts.prompt.PromptTemplate

In [22]:
type(llm)

langchain_groq.chat_models.ChatGroq

In [23]:
chain = prompt_extract | llm

In [24]:
response = chain.invoke({"page_data": page_data})

In [25]:
print(response.content)

{"role": "Senior AI Specialist", "experience": "Proven experience delivering AI solutions in a commercial environment", "skills": ["Advanced Analytics", "Artificial Intelligence", "Computer Vision", "Customer Experience Management", "Natural Language Processing", "Python"], "description": "Deliver end-to-end AI solutions from concept through to deployment and optimisation, shape AI strategies and roadmaps aligned to client objectives, design and build ML, NLP, Generative AI and advanced analytics models"}


In [31]:
type(response.content)

str

In [33]:
from langchain_core.output_parsers import JsonOutputParser

In [36]:
json_parser = JsonOutputParser()

In [37]:
json_response = json_parser.parse(response.content)

In [38]:
print(json_response)

{'role': 'Senior AI Specialist', 'experience': 'Proven experience delivering AI solutions in a commercial environment', 'skills': ['Advanced Analytics', 'Artificial Intelligence', 'Computer Vision', 'Customer Experience Management', 'Natural Language Processing', 'Python'], 'description': 'Deliver end-to-end AI solutions from concept through to deployment and optimisation, shape AI strategies and roadmaps aligned to client objectives, design and build ML, NLP, Generative AI and advanced analytics models'}
